<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/Glasgow_CNNLSTM_twoclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada sadece range-doppler ve spectrogram datası encoder decoder yapısı ile  Bi-LSTM ile train edildi.
# augmentation kaldırıldı



In [12]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:



# Load spectrogram_fall_dcnn fall data
spectrogram_fall_cnnlstm = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_fall_cnnlstm.mat')
spectrogram_fall_cnnlstm = spectrogram_fall_cnnlstm['spectrogram_fall_cnnlstm']
spectrogram_fall_cnnlstm = np.transpose(spectrogram_fall_cnnlstm, (2, 0, 1)) # label x width x height
spectrogram_fall_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_fall_label.mat')
spectrogram_fall_label = spectrogram_fall_label['spectrogram_fall_label']


# Load Spectrogram walking data
spectrogram_walking_cnnlstm= scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_walking_cnnlstm.mat')
spectrogram_walking_cnnlstm = spectrogram_walking_cnnlstm['spectrogram_walking_cnnlstm']
spectrogram_walking_cnnlstm = np.transpose(spectrogram_walking_cnnlstm, (2, 0, 1)) # label x width x height
spectrogram_walking_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_walking_label.mat')
spectrogram_walking_label = spectrogram_walking_label['spectrogram_walking_label']



In [4]:
spectrogram_concat_label = np.concatenate((spectrogram_fall_label,spectrogram_walking_label),axis=0)

# One-Hot Encode Label Spectrogram
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(spectrogram_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
spectrogram_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)



/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# Concat spectrogram data

spectrogram_concat = np.concatenate((spectrogram_fall_cnnlstm,spectrogram_walking_cnnlstm),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]

# Shuffle concat spectrogram
shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data

spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle_onehotencoded = spectrogram_concat_label_onehotencoded[shuffle_indx,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]




In [6]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,spectrogram_concat_label_shuffle):
  # ---------------- Parameters ----------------

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx
  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = spectrogram_concat_label_shuffle[randomlist_for_validation_indx,:]
  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]

  # Rotate Augmentation
  # get slow and fast indexes of training data
  slow_indexes = np.where(spectrogram_training_labels == 0)[0]
  fast_indexes = np.delete(range(0, spectrogram_training_labels.shape[0]), slow_indexes)

  slow_spectrograms_train = spectrogram_training_data[slow_indexes,:,:,:]
  size_of_samples_slow = slow_spectrograms_train.shape[0]

  fast_spectrograms_train = spectrogram_training_data[fast_indexes,:,:,:]
  size_of_samples_fast = fast_spectrograms_train.shape[0]


  augmented_image_fast = fast_spectrograms_train
  augmented_image_slow = slow_spectrograms_train
  spectrograms_fast_label = np.ones((size_of_samples_fast,1))
  spectrograms_slow_label = np.zeros((size_of_samples_slow,1))

  spectrogram_training_data = np.concatenate((augmented_image_fast,augmented_image_slow),axis=0)
  spectrogram_training_labels = np.concatenate((spectrograms_fast_label,spectrograms_slow_label),axis=0)


  return (spectrogram_training_data,spectrogram_training_labels,\
     spectrogram_validation_data, spectrogram_validation_labels)


In [7]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [8]:

spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))

In [9]:
print(spectrogram_concat.shape)
print(spectrogram_concat_label_shuffle.shape)

(450, 150, 153, 1)
(450, 1)


In [14]:
t = time.time()
# ---------- Parameters ----------------
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 120
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 256
dense_unit_of_spectrogram_function = 8
decoder_dense_unit = 256
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del spectrogram_concat_shuffle_test
    del spectrogram_concat_shuffle_train
    del spectrogram_train_data

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(spectrogram_concat_shuffle,spectrogram_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (spectrogram_train_data,spectrogram_train_label,\
     spectrogram_validation_data, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,spectrogram_concat_label_shuffle_train)



    # One-Hot Encode Label Train
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_train_label)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    training_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

        # One-Hot Encode Label Validation
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_validation_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    validation_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)


       # One-Hot Encode Label Test
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_concat_label_shuffle_test)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    test_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)





    # ---------------- Neural Network Architecture ----------------
    model = Sequential()
    model.add(Conv1D(filters=5, kernel_size=256, padding='same', activation='relu',input_shape=(153,150)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=3, kernel_size=512, padding='same', activation='relu',input_shape=(153,150)))
    model.add(LSTM(256))
    model.add(layers.Dense(2, activation='softmax'))
    opt = tf.keras.optimizers.Adam(0.0001)
    model.summary()


    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')

    print(spectrogram_train_data.shape)
    print(training_labels_onehotencoded.shape)



    history = model.fit(( spectrogram_train_data),(training_labels_onehotencoded),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((spectrogram_validation_data) , (validation_labels_onehotencoded)))
    test_loss, test_accuracy  = model.evaluate([spectrogram_concat_shuffle_test],\
                                               [test_label_onehotencoded],
                  batch_size=batch_size)
    gc.collect()

    test_accuracy_per_fold.append(test_accuracy)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer u

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 153, 5)         │       192,005 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 76, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 76, 3)          │         7,683 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256)            │       266,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 466,442 (1.78 MB)

 Trainable params: 466,442 (1.78 MB)

...

 Trainable params: 466,442 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

(288, 153, 150, 1)
(288, 2)
Epoch 1/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.5433 - loss: 0.6897 - val_accuracy: 0.7361 - val_loss: 0.5636
Epoch 2/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7873 - loss: 0.5020 - val_accuracy: 0.7778 - val_loss: 0.4721
Epoch 3/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8868 - loss: 0.2983 - val_accuracy: 0.9444 - val_loss: 0.1243
Epoch 4/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9605 - loss: 0.1241 - val_accuracy: 0.9722 - val_loss: 0.0844
Epoch 5/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9609 - loss: 0.0764 - val_accuracy: 0.9722 - val_loss: 0.0636
Epoch 6/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9706 - loss: 0.0897 - val_accuracy: 0.9861 - val_loss: 0.0591
Epoch 7/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9541 - loss: 0.1006 - val_accuracy: 0.9722 - val_loss: 0.0710
Epoch 8/120
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9525 - loss: 0.0873 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(None, 153, 150, 1))',)
  warnings.warn(msg)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9772 - loss: 0.0506 
[0.9822221994400024, 0.9844444274902344, 0.9733333230018616]
Mean test accuracy is 0.980, max test accuracy is 0.984, min test accuracy is 0.973,  std of test accuracy is 0.005.
Time elapsed through all process: 212.007, sec


In [15]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')

Mean test accuracy is 0.980, max test accuracy is 0.984, min test accuracy is 0.973,  std of test accuracy is 0.005.
